# Case - Machine Learning Engineer - Criação de Modelo

Este trabalho servirá de base para criação do modelo para previsão de atrasos de vôos, que será usado como case de uso de dados para vaga de Machine Learning Engineer. O projeto de análise dos dados está no arquivo `Notebook.ipynb`, disponível neste repositório. Como será apenas para ilustrar o processo, não serão feitas análises profundas dos dados. A proposta deste modelo será prever o atraso na chegada de um vôo (`arr_delay`) baseado em outros dados dos vôos

## Import de bibliotecas e dados

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnan, col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.functions import vector_to_array
import os
import pickle

In [60]:
spark=SparkSession.builder.config(
    "spark.driver.bindAddress", "localhost"
    ).config(
        "spark.network.timeout","10000000"
    ).config(
        "spark.executor.heartbeatInterval", "10000000"
    ).appName("MLE Case").getOrCreate()

In [61]:
df = spark.read.csv('./airports_database_extended.csv', header=True, inferSchema=True)

df.show()

+----+----------+------+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+--------+--------+----+------+-------------------+--------------------+-------------------+-----------------+---------------+
|dest|      date|origin| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|air_time|distance|hour|minute|          time_hour|                name|        date_as_str|wind_speed_origin|wind_speed_dest|
+----+----------+------+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+--------+--------+----+------+-------------------+--------------------+-------------------+-----------------+---------------+
| IAH|2013-01-01|   EWR|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|2013-01-01 00:0

In [62]:
df.describe().show()

+-------+------+------+-----------------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-------+------------------+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|summary|  dest|origin|               id|  year|             month|              day|          dep_time|    sched_dep_time|         dep_delay|          arr_time|   sched_arr_time|        arr_delay|carrier|            flight|tailnum|          air_time|          distance|              hour|            minute|                name| wind_speed_origin|   wind_speed_dest|
+-------+------+------+-----------------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-------+------------------+-------+------------------+-------------

Selecionando variáveis numéricas para análise de correlação.

In [63]:
selected_df = df.select(
    'dep_time', 'sched_dep_time', 'dep_delay', 'sched_arr_time', 'distance', 'wind_speed_origin', 'wind_speed_dest', 'arr_delay'
)

selected_df.show()

+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+
|dep_time|sched_dep_time|dep_delay|sched_arr_time|distance|wind_speed_origin|wind_speed_dest|arr_delay|
+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+
|   517.0|           515|      2.0|           819|    1400|             19.4|           19.1|     11.0|
|   533.0|           529|      4.0|           830|    1416|             22.5|           19.1|     20.0|
|   542.0|           540|      2.0|           850|    1089|             22.5|           23.2|     33.0|
|   544.0|           545|     -1.0|          1022|    1576|             22.5|           16.5|    -18.0|
|   554.0|           600|     -6.0|           837|     762|             22.5|           13.8|    -25.0|
|   554.0|           558|     -4.0|           728|     719|             19.4|           14.8|     12.0|
|   555.0|           600|     -5.0|           854|    1065|     

## Análise de correlação de variáveis

In [64]:
selected_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in selected_df.columns]).show()

+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+
|dep_time|sched_dep_time|dep_delay|sched_arr_time|distance|wind_speed_origin|wind_speed_dest|arr_delay|
+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+
|    8255|             0|     8255|             0|       0|                0|            254|     9430|
+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+



In [65]:
selected_df.count()

336776

Dada a quantidade de valores nulos em relação ao universo de dados, é seguro remover as linhas com valores nulos dos dados.

In [66]:
cleaned_df = selected_df.dropna()

cleaned_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cleaned_df.columns]).show()

+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+
|dep_time|sched_dep_time|dep_delay|sched_arr_time|distance|wind_speed_origin|wind_speed_dest|arr_delay|
+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+
|       0|             0|        0|             0|       0|                0|              0|        0|
+--------+--------------+---------+--------------+--------+-----------------+---------------+---------+



In [67]:
cleaned_df.coalesce(1).write.csv('./cleaned_dataset.csv', header=True, mode='overwrite')

In [68]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import pandas as pd

assembler = VectorAssembler(inputCols=cleaned_df.columns, outputCol='corr_features')
df_vector = assembler.transform(cleaned_df).select('corr_features')

pearson_corr = Correlation.corr(df_vector, 'corr_features', method='pearson').collect()[0][0]

corr_matrix = pearson_corr.toArray().tolist()
corr_matrix_df = pd.DataFrame(data=corr_matrix, columns=cleaned_df.columns, index=cleaned_df.columns)
corr_matrix_df.style.background_gradient(cmap='coolwarm')


,dep_time,sched_dep_time,dep_delay,sched_arr_time,distance,wind_speed_origin,wind_speed_dest,arr_delay
dep_time,1.000000,0.954766,0.259762,0.784117,-0.015255,0.008701,-0.005661,0.232551
sched_dep_time,0.954766,1.000000,0.199092,0.780229,-0.014110,0.008104,-0.006484,0.174154
dep_delay,0.259762,0.199092,1.000000,0.160643,-0.021719,0.036042,0.028489,0.914911
sched_arr_time,0.784117,0.780229,0.160643,1.000000,0.072480,0.007940,-0.016481,0.133491
distance,-0.015255,-0.014110,-0.021719,0.072480,1.000000,-0.006084,-0.037328,-0.061870
wind_speed_origin,0.008701,0.008104,0.036042,0.007940,-0.006084,1.000000,0.234047,0.060788
wind_speed_dest,-0.005661,-0.006484,0.028489,-0.016481,-0.037328,0.234047,1.000000,0.050715
arr_delay,0.232551,0.174154,0.914911,0.133491,-0.061870,0.060788,0.050715,1.000000


Verifica-se que o atraso de partida está diretamente correlacionado com o atraso de chegada, enquanto que a velocidade do vento e a distância, pela correlação de Pearson, aparenta não ter nenhuma relação com o atraso de chegada. No entanto, algumas outras variáveis estão diretamente correlacionadas entre si. Como este modelo é um modelo ilustrativo, todas as variáveis escolhidas serão usadas para estimar o atraso de chegada.

## Criação do Modelo

O modelo escolhido será um modelo de regressão linear múltipla. A escolha dessa abordagem é dada devido à sua simplicidade e robustez, que podem trazer bons resultados com um custo computacional relativamente baixo.

In [69]:
assembler = VectorAssembler(inputCols=[
    'dep_time', 'sched_dep_time', 'dep_delay', 'sched_arr_time',
    'distance', 'wind_speed_origin', 'wind_speed_dest'
    ],
    outputCol='features'
)

assembler_data = assembler.transform(cleaned_df)
model_df = assembler_data.select('features', 'arr_delay')

train_df, test_df = model_df.randomSplit([0.75, 0.25], seed=42)

In [71]:
test_df.show()

+--------------------+---------+
|            features|arr_delay|
+--------------------+---------+
|[1.0,2359.0,2.0,4...|      2.0|
|[3.0,1946.0,257.0...|    258.0|
|[3.0,2159.0,124.0...|    114.0|
|[3.0,2250.0,73.0,...|     75.0|
|[4.0,2359.0,5.0,4...|    -15.0|
|[6.0,2300.0,66.0,...|     64.0|
|[7.0,2359.0,8.0,4...|     16.0|
|[10.0,2137.0,153....|    171.0|
|[10.0,2359.0,11.0...|     -4.0|
|[11.0,2229.0,102....|     93.0|
|[11.0,2359.0,12.0...|     -9.0|
|[11.0,2359.0,12.0...|     -8.0|
|[14.0,2359.0,15.0...|     18.0|
|[16.0,2111.0,185....|    208.0|
|[16.0,2159.0,137....|    130.0|
|[16.0,2359.0,17.0...|     33.0|
|[16.0,2359.0,17.0...|      3.0|
|[17.0,2359.0,18.0...|      6.0|
|[22.0,2159.0,143....|    128.0|
|[27.0,2035.0,232....|    220.0|
+--------------------+---------+
only showing top 20 rows


In [54]:
lin_reg = LinearRegression(featuresCol='features', labelCol='arr_delay', predictionCol='predicted_arr_delay')
lin_reg_model = lin_reg.fit(train_df)

25/10/25 20:07:02 WARN Instrumentation: [1209288a] regParam is zero, which might cause numerical instability and overfitting.


## Teste do modelo

In [55]:
pred = lin_reg_model.transform(test_df)

eval_rmse = RegressionEvaluator(labelCol='arr_delay', predictionCol='predicted_arr_delay', metricName='rmse')
rmse = eval_rmse.evaluate(pred)

eval_r2 = RegressionEvaluator(labelCol='arr_delay', predictionCol='predicted_arr_delay', metricName='r2')
r2 = eval_r2.evaluate(pred)

print(f"Métrica RMSE: {rmse}")
print(f"Métrica R2: {r2}")

Métrica RMSE: 17.82709415236725
Métrica R2: 0.8403795009534463


Temos então um RMSE de 17.82, e um R2 de 84%, o que indica um bom desempenho do modelo, apesar de não ser ótimo. Algumas outras abordagens poderiam ser tomadas, como normalização de variáveis, adicionar variáveis categóricas discretizadas, que poderiam aumentar a eficácia do modelo. No entanto, somente com essa abordagem, podemos seguir para os próximos passos do case. Vamos salvar o modelo para prosseguir.

## Salvando modelo em formato .pkl

In [56]:
coef = lin_reg_model.coefficients
intercept = lin_reg_model.intercept
feat = lin_reg_model.summary.featuresCol

model_params = {
    'coefficients': coef,
    'intercept': intercept,
    'features': feat
}

model_name = 'linreg_model_case_params.pkl'

model_path = os.path.join(os.path.dirname(os.getcwd()), 'src', 'model')

model_path = os.path.join(model_path, model_name)

with open(model_path, 'wb') as file:
    pickle.dump(model_params, file)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

linreg_sklearn = LinearRegression()

coef_toarray = coef.toArray()

linreg_sklearn.coef_ = np.array(coef_toarray)
linreg_sklearn.intercept_ = intercept

linreg_sklearn.n_features_in_ = len(coef_toarray)

In [58]:
model_name = 'linreg_model_sklearn.pkl'

model_path = os.path.join(os.path.dirname(os.getcwd()), 'src', 'model')

model_path = os.path.join(model_path, model_name)

with open(model_path, 'wb') as file:
    pickle.dump(linreg_sklearn, file)